### P1

#### cpp-p1.cpp

``` cpp
#include <iostream>
#include <math.h>
#include <string>
#include "./gnuplot-iostream.h"

using namespace std;


int main(){

    // We initilize arrays for our variables
    double x_array[100];
    double y_array[100];
    double z_array[100][100];

    // Using gnuplot for plotting
    Gnuplot gp1;
    
    // First we plot y = Sin(2x)
    // Calculating the function for each point
    for (int i=0; i<100; i++){
        x_array[i] = double(i) / 20.0;
        y_array[i] = sin(2.0*x_array[i]);

    }

    // Ploting and exportnig
    gp1 << "set xrange[0:5]\n set yrange[-1.5:1.5] \n";
    gp1 << "set size square \n";
    gp1 << "set title " << "\"y = Sin(2x) \"" <<"\n";
    gp1 << "set output " << "\"sin-plot.png\"" << "\n"; // Comment this line and the next for plot only mode
    gp1 << "set terminal pngcairo size 8000, 8000 fontscale 10 linewidth 10 pointscale 1\n";
    gp1 << "plot '-' with lines title \"\" \n";

    for (int i=0; i<100; i++){
        gp1 << to_string(x_array[i]) << " " << to_string(y_array[i]) << "\n";
    }

    // gp << "clear";

    // Now we calculate and plot z = Tanh(x*y)
    for (int i=0; i<100; i++){
        x_array[i] = double(i) / 20.0 - 2.5;
        for (int j=0; j<100; j++){
                y_array[j] = double(j) / 20.0 - 2.5;
                z_array[i][j] = tanh(x_array[i]*y_array[j]);
        }

    }



    // Ploting and exportnig
    Gnuplot gp2;
    gp2 << "set xrange[-2.5:2.5]\n set yrange[-2.5:2.5] \n";
    gp2 << "set size square \n";
    gp2 << "set title " << "\"z = Tanh(x*y) \"" <<"\n";
    gp2 << "set output " << "\"surface-plot.png\"" << "\n"; // Comment this line and the next for plot only mode
    gp2 << "set terminal pngcairo size 8000, 8000 fontscale 10 linewidth 10 pointscale 1\n";
    gp2 << "set hidden3d \n ";
    gp2 << "splot '-' with lines lc rgb \'#b90046\' lw 1.5 title \"\"\n";

    for (int i=0; i<100; i++){
        for (int j=0; j<100; j++){
            gp2 << to_string(x_array[i]) << " " << to_string(y_array[j]) << " " << to_string(z_array[i][j]) << "\n";
        }
        gp2 << "\n";
    }

    

    return 0;
}

```

#### Results

![Alt text](C++-code/p1/sin-plot.png)

![Alt text](C++-code//p1/surface-plot.png)

### P2

#### cpp-p2.cpp

``` cpp
#include <iostream>
#include <ctime> 
#include <math.h>
#include <string>
#include <complex>
#include "./gnuplot-iostream.h"

using namespace std;

extern "C" void zheev_(char*, char*, int*, complex<double>*, int*, double*, complex<double>*, int*, double*, int*);
void calc_eigen(complex<double>* mat, int N_mat, double* eigen_out);

int main(){

    srand((unsigned)time(0)); // Seeding the random generator

    Gnuplot gp1;

    // We initilize arrays for our variables
    const int N = 500;
    complex<double> A[N][N];
    complex<double> H_sum[N][N];
    double eigenvals[N];
    double x_r, x_i;


    for (int i=0; i<N; i++){
        for (int j=0; j<N; j++){

            x_i = double(rand())/double(RAND_MAX);
            x_r = double(rand())/double(RAND_MAX);
            A[i][j] = complex<double> {x_i, x_r};
        }    
    }    

    // Adding the conjugate transpose manually (there are better ways but we ignore them for now)
    for (int i=0; i<N; i++){
        for (int j=0; j<N; j++){

            H_sum[i][j] = A[i][j] + conj(A[j][i]);
        }    
    }  

    // Computing the eigenvalues and storing them in "eigenvals"
    calc_eigen(H_sum[0], N, eigenvals);

    // Ploting and exportnig
    gp1 << "set style fill solid border -1 \n";
    gp1 << "set key off \n";
    gp1 << "set title " << "\"A + A'  Eigenvalues \"" <<"\n";
    gp1 << "set key off "<<"\n";
    gp1 << "binwidth=1.5 "<<"\n";
    gp1 << "bin(x,width)=width*floor(x/width) "<<"\n";
    gp1 << "set output " << "\"hist-sum.png\"" << "\n"; // Comment this line and the next for plot only mode
    gp1 << "set terminal pngcairo size 2000, 2000 fontscale 4\n";
    gp1 << "plot '-' using (bin($1,binwidth)):(1.0) smooth freq with boxes \n";

    for (int i=0; i<N-1; i++){
        gp1 << to_string(eigenvals[i])<< "\n";
    }

    return 0;
}

void calc_eigen(complex<double>* mat, int N_mat, double* eigen_out){

    // Calculates eigenvalues of hermitian matrices
    // This functions uses the ZHEEV function from LAPACK lib.

    int LWORK = 6*N_mat;
    double RWORK[3*N_mat - 2];
    complex<double> WORK[6*N_mat];
    int info;

    char par1 = 'N';
    char par2 = 'U';

    zheev_(&par1, &par2, &N_mat, mat, &N_mat, eigen_out, WORK, &LWORK, RWORK, &info);

    cout<<"ZHEEV info: "<<info<<"\n";

}
```

#### Results

![Alt text](C++-code/p2/hist-sum.png)